In [1]:
!pip install langchain
!pip install langchain_community
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.1 MB/s eta 0:00:00


In [2]:
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
import os
from langchain_community.vectorstores import FAISS

In [3]:
def extract_data(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    docs = loader.load()

    return docs

extracted = extract_data("/content/drive/MyDrive/Colab Notebooks/Medical_Chatbot/Data")


In [4]:
#Create chunks of data
def doc_spliter(extracted):
    chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
    text_chunks = chunks.split_documents(extracted)

    return text_chunks

documents_split = doc_spliter(extracted)

In [6]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [12]:
#Embedding model
def embeddings_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device':'cpu'} )

    return embeddings

embeddings = embeddings_model()

In [13]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.4 MB/s eta 0:00:00


In [14]:
#Creating vectors

vector_store = FAISS.from_documents(documents_split, embeddings)
vector_store.save_local("/content/drive/MyDrive/Colab Notebooks/Medical_Chatbot/vectorestores")


In [15]:
#Prompt template
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompts = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt":prompts}

In [ ]:
#LLM Model(Llama-2)
llm = CTransformers(model="/content/drive/MyDrive/Colab Notebooks/Medical_Chatbot/Data/llama-2-7b-chat.ggmlv3.q8_0.bin",
                    model_type="llama",
                    config={"max_new_tokens":512,
                            "temperature":0.8})


In [ ]:
#QA retrieval
vector_data = "/content/drive/MyDrive/Colab Notebooks/Medical_Chatbot/vectorestores"
QA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_data.as_retriever(search_kwargs={"k":3}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
#Testing
while True:
    user_input = input(f"Input Prompt:")
    result = QA({"query":user_input})
    print("Response: ", result["result"])
